<a href="https://colab.research.google.com/github/AndresChavez123/SenalesySistemas/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalación de librerías
!pip install streamlit -q
!pip install --upgrade control -q
!pip install soundfile yt-dlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.3/578.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.1 MB/s eta 0:00:00


In [ ]:
!apt install ffmpeg -y
!mkdir pages

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
%%writefile 📝_Proyecto_Final.py

import streamlit as st

# Configuración de la app
st.set_page_config(
    page_title="Explorador de Señales – UNAL",
    page_icon="📶",
    layout="wide"
)

# Título principal
st.title("📶 Explorador Interactivo de Señales – UNAL 2025")
st.markdown("## _Conectando Teoría y Tecnología: Del análisis temporal a la comunicación digital_")
st.markdown("---")

# =============================
# Bienvenida
# =============================
st.markdown("""
🙌 **¡Hola!**

Este entorno ha sido creado como una **plataforma de exploración interactiva** para el proyecto final del curso de **Señales y Sistemas**, con el fin de visualizar de manera intuitiva cómo se procesan y transmiten señales en los sistemas modernos de comunicación.

---

### 🔍 ¿De qué se trata este proyecto?

A través de gráficos dinámicos, controles deslizantes y ejemplos reales, podrás:

- Analizar cómo una señal se transforma entre distintos dominios.
- Comprender los fundamentos del procesamiento digital.
- Simular tecnologías de modulación avanzadas como **QAM y OFDM**.
- Explorar cómo estas técnicas dan vida a tecnologías inalámbricas como **WiFi** y **5G**.

---

### 🧩 Secciones principales del dashboard

Cada sección del menú lateral presenta una **pieza clave** del rompecabezas que compone la comunicación moderna:

1. **Análisis espectral** – Fourier y la descomposición de frecuencias.
2. **Procesamiento digital** – Filtros para atenuar o preservar contenido útil.
3. **Componentes I/Q** – Señales analíticas y cuadratura de fase.
4. **QAM** – Transmisión de información en el plano complejo.
5. **OFDM y sistemas reales** – Cómo se estructura la transmisión digital actual.
6. **Visualización global** – Construcción y evaluación completa de una señal procesada.

---

### 📘 Información del curso

- **Asignatura:** Señales y Sistemas – 2025
- **Docente:** Dr. Andrés Marino Álvarez Meza
- **Universidad:** Nacional de Colombia – Sede Manizales
- **Equipo de desarrollo:** Juan Esteban Ramírez Montoya -
Juan Esteban Osorio Gonzalez -
Andres Mauricio Chavez

---

Disfruta la experiencia interactiva y no olvides experimentar con todos los parámetros. Cada módulo está diseñado para ayudarte a **visualizar conceptos abstractos de forma concreta y didáctica**.
""")


Writing 📝_Proyecto_Final.py


In [ ]:
%%writefile _Analisis_Comp_Senales.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from scipy.signal.windows import blackman
from numpy.fft import fft, fftfreq

# Configuración inicial de la interfaz
st.set_page_config(page_title="Exploración de Señales", page_icon="📊")
st.title("📊 Explorador Espectral y Filtro Digital de Señales Artificiales")

st.markdown("""
Este espacio interactivo permite construir, analizar y modificar señales compuestas. Podrás:

- Generar señales con múltiples frecuencias.
- Visualizar su representación temporal.
- Observar la descomposición espectral mediante FFT.
- Aplicar un filtro digital tipo Butterworth.
- Analizar la respuesta en frecuencia del sistema.

---
""")

# === Controles laterales ===
st.sidebar.header(" Generador de Señal")
fs = st.sidebar.slider("Tasa de Muestreo [Hz]", 1000, 6000, 2000, 500)
T = st.sidebar.slider("Duración total [seg]", 0.5, 3.0, 1.0, 0.1)

f_a = st.sidebar.slider("Tono 1 [Hz]", 10, fs//2 - 1, 40, 1)
f_b = st.sidebar.slider("Tono 2 [Hz]", 10, fs//2 - 1, 120, 1)
f_c = st.sidebar.slider("Tono 3 [Hz]", 10, fs//2 - 1, 300, 1)

# === Síntesis de señal ===
t = np.linspace(0, T, int(fs*T), endpoint=False)
s = (np.sin(2*np.pi*f_a*t) +
     0.6 * np.sin(2*np.pi*f_b*t) +
     0.4 * np.sin(2*np.pi*f_c*t))

st.subheader(" Dominio Temporal")
st.markdown(f"Se construyó una señal de `{T}` segundos con muestreo `{fs} Hz`, combinando frecuencias `{f_a}`, `{f_b}` y `{f_c}` Hz.")

fig0, ax0 = plt.subplots()
ax0.plot(t, s)
ax0.set_xlabel("Tiempo [s]")
ax0.set_ylabel("Amplitud")
ax0.set_title("Señal Compuesta (Tiempo)")
ax0.grid(True)
st.pyplot(fig0)

# === FFT de señal original ===
st.subheader(" Espectro de la Señal (FFT)")
st.markdown("Aplicamos FFT tras enventanar con una función Blackman para mejorar la resolución espectral.")

win = blackman(len(s))
s_w = s * win
S_fft = fft(s_w)
f_spec = fftfreq(len(t), 1/fs)
mag_spec = 20 * np.log10(np.abs(S_fft) + 1e-10)

fig1, ax1 = plt.subplots()
ax1.plot(f_spec[:len(f_spec)//2], mag_spec[:len(mag_spec)//2])
ax1.set_xlabel("Frecuencia [Hz]")
ax1.set_ylabel("Magnitud [dB]")
ax1.set_title("Espectro (Blackman + FFT)")
ax1.grid()
st.pyplot(fig1)

# === Filtro digital configurado ===
st.sidebar.header("🔻 Configura el Filtro Digital")
cut = st.sidebar.slider("Frecuencia de Corte [Hz]", 20, fs//2 - 5, 150, 5)
n_order = st.sidebar.slider("Orden del Filtro", 2, 10, 5)

def diseño_filtro(fc, fs, orden):
    nyquist = 0.5 * fs
    norm = fc / nyquist
    b, a = butter(orden, norm, btype='low', analog=False)
    return b, a

b, a = diseño_filtro(cut, fs, n_order)
filt_out = lfilter(b, a, s)

st.subheader(" Resultado del Filtrado")
st.markdown(f"Se aplicó un filtro digital de **orden `{n_order}`** con corte en `{cut} Hz` para eliminar componentes altas.")

fig2, ax2 = plt.subplots()
ax2.plot(t, s, '--', label="Original", alpha=0.5)
ax2.plot(t, filt_out, label="Filtrada", linewidth=2)
ax2.set_xlabel("Tiempo [s]")
ax2.set_ylabel("Amplitud")
ax2.set_title("Señal Antes y Después del Filtro")
ax2.legend()
ax2.grid()
st.pyplot(fig2)

# === FFT de la señal filtrada ===
st.subheader(" Análisis Frecuencial Post-Filtro")
S_filt_fft = fft(filt_out * blackman(len(filt_out)))
mag_filt = 20 * np.log10(np.abs(S_filt_fft) + 1e-10)

fig3, ax3 = plt.subplots()
ax3.plot(f_spec[:len(f_spec)//2], mag_filt[:len(mag_filt)//2])
ax3.set_xlabel("Frecuencia [Hz]")
ax3.set_ylabel("Magnitud [dB]")
ax3.set_title("FFT Tras Filtrado")
ax3.grid()
st.pyplot(fig3)

# === Diagrama de Bode ===
st.subheader(" Respuesta en Frecuencia del Filtro")
w, h = freqz(b, a, worN=2048)
frecs = 0.5 * fs * w / np.pi
gain = 20 * np.log10(np.abs(h))

fig4, ax4 = plt.subplots()
ax4.semilogx(frecs, gain)
ax4.set_xlabel("Frecuencia [Hz]")
ax4.set_ylabel("Ganancia [dB]")
ax4.set_title("Diagrama de Bode del Filtro Digital")
ax4.axvline(cut, color='red', linestyle='--', label=f"Corte: {cut} Hz")
ax4.grid(True, which="both", ls="--")
ax4.legend()
st.pyplot(fig4)

st.markdown("---")
st.info(" Modifica los parámetros para observar cómo cambian la señal y su espectro.")


Writing _Analisis_Comp_Senales.py


In [ ]:
!mv _Analisis_Comp_Senales.py pages/

In [ ]:
%%writefile analisis_IQ_streamlit.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, butter, freqz, lfilter
from scipy.signal.windows import hann
from scipy.fft import fft, fftfreq, fftshift
from scipy.io import wavfile
import subprocess
import yt_dlp as youtube_dl

st.set_page_config(page_title="🧿 Señales en I/Q", page_icon="🎧", layout="wide")
st.title("🎧 Exploración de Componentes I/Q mediante Hilbert")

st.markdown("""
## 🧭 Propósito

Estudiar una señal acústica real en su forma **compleja** utilizando la **Transformada de Hilbert**, con el objetivo de observar sus partes **en fase ($I(t)$)** y **en cuadratura ($Q(t)$)**.
Estas variables son claves en esquemas de modulación digital como **QAM**, presentes en tecnologías como 5G y comunicaciones satelitales.

---

## 🧩 Concepto Clave

Dada una señal real $x(t)$, su extensión compleja o **señal analítica** es:

$$
x_a(t) = x(t) + j \cdot \mathcal{H}\{x(t)\} = I(t) + jQ(t)
$$

Donde:
- $I(t) = x(t)$: señal original (fase).
- $Q(t) = \mathcal{H}\{x(t)\}$: versión en cuadratura (90° fuera de fase).

Este formato complejo permite representar la información como puntos en el **plano complejo**, típico en comunicación digital.

---
""")

# ==== Configuración lateral ====
st.sidebar.header("🔧 Ajustes del análisis")
frecuencia = st.sidebar.slider("Tasa de muestreo (Hz)", 8000, 44100, 16000, 1000)
tiempo_max = st.sidebar.slider("Duración máxima (s)", 1, 10, 3, 1)
youtube_url = st.sidebar.text_input("🎥 Pega un enlace de YouTube para obtener audio:")

# ==== Extracción de audio ====
cargado = False
if youtube_url:
    try:
        with st.spinner("🎶 Extrayendo contenido desde YouTube..."):
            ydl_opts = {'format': 'bestaudio', 'outtmpl': 'audio_temp.%(ext)s'}
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download([youtube_url])

            subprocess.run(['ffmpeg', '-y', '-i', 'audio_temp.webm',
                            '-ar', str(frecuencia), '-ac', '1', 'temp.wav'],
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

            sr, señal = wavfile.read('temp.wav')
            señal = señal.astype(np.float32)
            señal /= np.max(np.abs(señal))
            señal = señal[:int(frecuencia * tiempo_max)]
            tiempo = np.linspace(0, len(señal)/frecuencia, len(señal), endpoint=False)
            cargado = True
    except Exception as e:
        st.error(f"⚠️ No se pudo procesar el audio: {e}")

# ==== Análisis I/Q ====
if cargado:
    st.markdown("## 🔬 Componentes I(t) y Q(t)")

    señal_q = np.imag(hilbert(señal))

    fig1, axs1 = plt.subplots(2, 1, figsize=(10, 4), sharex=True)
    axs1[0].plot(tiempo, señal, label="I(t)")
    axs1[1].plot(tiempo, señal_q, color='darkorange', label="Q(t)")
    axs1[0].set_title("🔵 Señal en fase")
    axs1[1].set_title("🟠 Señal en cuadratura")
    axs1[0].legend(); axs1[1].legend()
    axs1[0].grid(); axs1[1].grid()
    st.pyplot(fig1)

    st.markdown("""
### ℹ️ Nota técnica

La señal $Q(t)$ es la **transformada de Hilbert** de $I(t)$, desplazada 90°.
Juntas forman un par ortogonal que puede representarse como puntos complejos en modulaciones digitales.

---
""")

    st.markdown("## 📍 Representación en el plano I/Q")
    fig2, ax2 = plt.subplots(figsize=(6, 4))
    ax2.scatter(señal, señal_q, s=1, alpha=0.5)
    ax2.set_xlabel("I(t)"); ax2.set_ylabel("Q(t)")
    ax2.set_title("Nube de puntos complejos"); ax2.grid()
    st.pyplot(fig2)

    st.markdown("## 📊 Espectro con ventana de Hann")
    N = len(señal)
    w_hann = hann(N)
    señal_win = señal * w_hann
    señalq_win = señal_q * w_hann

    f = fftshift(fftfreq(N, 1/frecuencia))
    fft_i = fftshift(np.abs(fft(señal_win)) / np.sum(w_hann))
    fft_q = fftshift(np.abs(fft(señalq_win)) / np.sum(w_hann))

    fig3, ax3 = plt.subplots(figsize=(10, 3))
    ax3.plot(f, fft_i, label="|FFT(I)|")
    ax3.plot(f, fft_q, linestyle='--', label="|FFT(Q)|")
    ax3.set_xlim(0, frecuencia//2)
    ax3.set_xlabel("Frecuencia [Hz]"); ax3.set_ylabel("Magnitud")
    ax3.set_title("Análisis espectral")
    ax3.legend(); ax3.grid()
    st.pyplot(fig3)

    st.markdown("## 📉 Respuesta de filtro Butterworth")
    b, a = butter(4, 0.3)
    w, h = freqz(b, a, worN=8000)
    fase = np.unwrap(np.angle(h))

    fig_b, (ax_mag, ax_fase) = plt.subplots(2, 1, figsize=(8, 4))
    ax_mag.plot(frecuencia * w / (2*np.pi), 20*np.log10(np.abs(h)))
    ax_mag.set_title("Magnitud del filtro"); ax_mag.set_ylabel("dB"); ax_mag.grid()

    ax_fase.plot(frecuencia * w / (2*np.pi), fase * 180/np.pi)
    ax_fase.set_title("Fase desenrollada"); ax_fase.set_xlabel("Frecuencia [Hz]")
    ax_fase.set_ylabel("°"); ax_fase.grid()
    st.pyplot(fig_b)

    st.markdown("## 📏 Envolvente y Fase Instantánea")
    analitica = señal + 1j * señal_q
    modulo = np.abs(analitica)
    angulo = np.angle(analitica)
    mascara = modulo > 0.01

    fig4, ax4 = plt.subplots(2, 1, figsize=(10, 4), sharex=True)
    ax4[0].plot(tiempo, modulo, color='purple')
    ax4[1].plot(tiempo[mascara], angulo[mascara], color='darkgreen')
    ax4[0].set_title("🔺 Envolvente"); ax4[1].set_title("🔻 Fase inst.")
    ax4[0].grid(); ax4[1].grid()
    st.pyplot(fig4)

    # === MODULACIÓN ===
    st.markdown("## 🚀 Modulación tipo QAM")
    f_portadora = st.sidebar.slider("⚙️ Frecuencia portadora (Hz)", 1000, frecuencia//2 - 500, 4000, 500)

    port_cos = np.cos(2 * np.pi * f_portadora * tiempo)
    port_sin = np.sin(2 * np.pi * f_portadora * tiempo)
    señal_modulada = señal * port_cos - señal_q * port_sin

    st.markdown("### 🧾 Señal en tiempo (QAM)")
    fig5, ax5 = plt.subplots(figsize=(10, 3))
    ax5.plot(tiempo[:1000], señal_modulada[:1000])
    ax5.set_title("Muestra de señal QAM"); ax5.set_xlabel("Tiempo [s]"); ax5.set_ylabel("Valor")
    ax5.grid()
    st.pyplot(fig5)

    st.markdown("### 📡 Frecuencia (QAM)")
    señal_mod_win = señal_modulada * hann(len(señal_modulada))
    fft_modulada = fftshift(np.abs(fft(señal_mod_win)) / np.sum(w_hann))
    fig6, ax6 = plt.subplots(figsize=(10, 3))
    ax6.plot(f, fft_modulada)
    ax6.set_title("Espectro de la señal QAM"); ax6.set_xlabel("Frecuencia [Hz]")
    ax6.set_ylabel("Magnitud"); ax6.set_xlim(0, frecuencia//2); ax6.grid()
    st.pyplot(fig6)

    # === RUIDO ===
    st.markdown("## 🌪️ Canal con Ruido Blanco")
    snr = st.sidebar.slider("📢 SNR [dB]", 0, 60, 30, 5)
    potencia = np.mean(señal_modulada**2)
    pot_ruido = potencia / (10**(snr/10))
    ruido = np.random.normal(0, np.sqrt(pot_ruido), size=señal_modulada.shape)
    señal_con_ruido = señal_modulada + ruido

    fig7, ax7 = plt.subplots(figsize=(10, 3))
    ax7.plot(tiempo[:1000], señal_con_ruido[:1000])
    ax7.set_title("Señal afectada por ruido"); ax7.set_xlabel("Tiempo"); ax7.set_ylabel("Amplitud")
    ax7.grid()
    st.pyplot(fig7)

    fft_ruido = fftshift(np.abs(fft(señal_con_ruido * hann(len(señal_con_ruido)))) / np.sum(w_hann))
    fig8, ax8 = plt.subplots(figsize=(10, 3))
    ax8.plot(f, fft_ruido)
    ax8.set_title("Espectro de señal ruidosa"); ax8.set_xlabel("Frecuencia"); ax8.set_ylabel("Magnitud")
    ax8.set_xlim(0, frecuencia//2); ax8.grid()
    st.pyplot(fig8)

    st.success("🔄 ¡Simulación completa con canal ruidoso!")

    # === DEMODULACIÓN ===
    st.markdown("## 🔁 Recuperación de Señales I y Q")
    señal_i_demod = señal_con_ruido * port_cos
    señal_q_demod = -señal_con_ruido * port_sin

    b_lp, a_lp = butter(4, 0.3)
    señal_i_rec = lfilter(b_lp, a_lp, señal_i_demod)
    señal_q_rec = lfilter(b_lp, a_lp, señal_q_demod)

    fig9, ax9 = plt.subplots(2, 1, figsize=(10, 4), sharex=True)
    ax9[0].plot(tiempo, señal, label="I original", alpha=0.6)
    ax9[0].plot(tiempo, señal_i_rec, linestyle='--', label="I recuperada")
    ax9[0].legend(); ax9[0].set_title("🔷 Comparación de I(t)"); ax9[0].grid()

    ax9[1].plot(tiempo, señal_q, label="Q original", alpha=0.6)
    ax9[1].plot(tiempo, señal_q_rec, linestyle='--', label="Q recuperada")
    ax9[1].legend(); ax9[1].set_title("🟠 Comparación de Q(t)"); ax9[1].grid()

    st.pyplot(fig9)

    st.success("✅ ¡Demodulación exitosa!")


Writing analisis_IQ_streamlit.py


In [ ]:
!mv analisis_IQ_streamlit.py pages/

In [ ]:
%%writefile qam_visualizer_tool.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift

# Configuración inicial de la app
st.set_page_config(page_title="🔗 Codificación 16-QAM", page_icon="📡", layout="wide")
st.title("📡 Explorador Interactivo de Señales 16-QAM")

st.markdown("""
## 🧭 Objetivo General

Este módulo explora la construcción y análisis de una señal digital usando **Modulación por Amplitud en Cuadratura (16-QAM)**.
Verás cómo se generan los componentes I/Q, su combinación, su contenido espectral, y la constelación digital resultante.

---

## 🧠 ¿Qué es 16-QAM?

El esquema 16-QAM utiliza combinaciones de amplitudes en dos ejes ortogonales (I y Q) para representar grupos de 4 bits.
Se representa como:

$$
x(t) = I(t)\cos(2\pi f_ct) - Q(t)\sin(2\pi f_ct)
$$

Donde:

- $I(t)$: componente en fase
- $Q(t)$: componente en cuadratura
- $f_c$: frecuencia de portadora

---
""")

# Parámetros de entrada
st.sidebar.header("🛠 Parámetros de Modulación")
f_c = st.sidebar.slider("🎚 Portadora [Hz]", 500, 8000, 2000, 500)
baud_rate = st.sidebar.slider("⏱ Velocidad de símbolo [baud]", 100, 4000, 1000, 100)
fs = st.sidebar.slider("🔍 Frecuencia de muestreo [Hz]", 10000, 100000, 40000, 5000)
symbols = st.sidebar.slider("🔢 Número de símbolos", 20, 1000, 200, 20)

# Generar flujo binario aleatorio y codificar
bits = np.random.randint(0, 2, 4 * symbols)
I_vals, Q_vals = [], []

mapa_16qam = {
    (0,0,0,0): (-3, +3), (0,0,0,1): (-3, +1),
    (0,0,1,1): (-3, -1), (0,0,1,0): (-3, -3),
    (0,1,1,0): (-1, -3), (0,1,1,1): (-1, -1),
    (0,1,0,1): (-1, +1), (0,1,0,0): (-1, +3),
    (1,1,0,0): (+1, +3), (1,1,0,1): (+1, +1),
    (1,1,1,1): (+1, -1), (1,1,1,0): (+1, -3),
    (1,0,1,0): (+3, -3), (1,0,1,1): (+3, -1),
    (1,0,0,1): (+3, +1), (1,0,0,0): (+3, +3),
}

for i in range(0, len(bits), 4):
    grupo = tuple(bits[i:i+4])
    I_val, Q_val = mapa_16qam[grupo]
    I_vals.append(I_val)
    Q_vals.append(Q_val)

I_vals = np.array(I_vals)
Q_vals = np.array(Q_vals)

# Tiempo y construcción de señal
Ts = 1 / baud_rate
samples_per_symb = int(fs * Ts)
t = np.arange(0, symbols * Ts, 1 / fs)

I_signal = np.repeat(I_vals, samples_per_symb)
Q_signal = np.repeat(Q_vals, samples_per_symb)
cos = np.cos(2 * np.pi * f_c * t)
sin = np.sin(2 * np.pi * f_c * t)
qam_tx = I_signal * cos - Q_signal * sin

# === Visualización temporal ===
st.markdown("## ⏱ Evolución de la señal en el tiempo")

fig1, axes = plt.subplots(3, 1, figsize=(10, 6), sharex=True)
axes[0].plot(t, I_signal, color='royalblue', label='I(t)')
axes[1].plot(t, Q_signal, color='firebrick', label='Q(t)')
axes[2].plot(t, qam_tx, color='forestgreen', label='QAM(t)')

axes[0].set_ylabel("I(t)")
axes[1].set_ylabel("Q(t)")
axes[2].set_ylabel("QAM(t)")
axes[2].set_xlabel("Tiempo [s]")

for ax in axes:
    ax.grid(True)
    ax.legend()
st.pyplot(fig1)

st.info("La señal QAM combina I y Q, moduladas con coseno y seno, respectivamente. Esto permite transmitir 4 bits por símbolo con alta eficiencia espectral.")

# === Espectro ===
st.markdown("## 🌐 Espectro de la Señal QAM")

N = len(qam_tx)
freqs = fftshift(fftfreq(N, d=1/fs))
spectrum = fftshift(np.abs(fft(qam_tx)) / N)

fig2, ax2 = plt.subplots(figsize=(10, 3))
ax2.plot(freqs, spectrum, color='purple')
ax2.set_xlim(0, fs/2)
ax2.set_title("Espectro de Potencia de la Señal")
ax2.set_xlabel("Frecuencia [Hz]")
ax2.set_ylabel("Magnitud")
ax2.grid()
st.pyplot(fig2)

st.caption("💬 El espectro se centra alrededor de la portadora. Mayor tasa de símbolos = mayor ancho de banda.")

# === Constelación ===
st.markdown("## 🌌 Mapa de Constelación")

fig3, ax3 = plt.subplots()
ax3.scatter(I_vals, Q_vals, alpha=0.7, color='darkmagenta', edgecolors='black')
ax3.set_xlabel("Componente I")
ax3.set_ylabel("Componente Q")
ax3.set_title("Constelación 16-QAM")
ax3.grid()
st.pyplot(fig3)

st.success("✔️ Visualización finalizada. ¡Explora diferentes configuraciones desde la barra lateral!")


Writing qam_visualizer_tool.py


In [ ]:
!mv qam_visualizer_tool.py pages/

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 📝_Proyecto_Final.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-07-24 00:11:51--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-07-24 00:11:51--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-24T01%3A08%3A51Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-24T0

In [11]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
